In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
df1 = pd.read_csv('202301_KRW-BTC_ohlcv.csv', index_col = 'index')
df2 = pd.read_csv('202302_KRW-BTC_ohlcv.csv', index_col = 'index')
df3 = pd.read_csv('202303_KRW-BTC_ohlcv.csv', index_col = 'index')
df4 = pd.read_csv('202304_KRW-BTC_ohlcv.csv', index_col = 'index')
df5 = pd.read_csv('202305_KRW-BTC_ohlcv.csv', index_col = 'index')
df6 = pd.read_csv('202306_KRW-BTC_ohlcv.csv', index_col = 'index')
df7 = pd.read_csv('202307_KRW-BTC_ohlcv.csv', index_col = 'index')
df8 = pd.read_csv('202308_KRW-BTC_ohlcv.csv', index_col = 'index')

In [3]:
df_2023 = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8])

In [4]:
df_2023.drop(['value'], axis=1, inplace=True)

In [5]:
df_2023['mid_price'] = df_2023['high']/2 + df_2023['low']/2

In [6]:
df_2023.head()

,open,high,low,close,volume,mid_price
index,,,,,,
2023-01-01 09:00:00,21079000.0,21082000.0,21061000.0,21081000.0,4.852014,21071500.0
2023-01-01 09:01:00,21080000.0,21080000.0,21061000.0,21061000.0,2.099329,21070500.0
2023-01-01 09:02:00,21062000.0,21062000.0,21055000.0,21062000.0,2.486127,21058500.0
2023-01-01 09:03:00,21057000.0,21078000.0,21055000.0,21064000.0,1.744549,21066500.0
2023-01-01 09:04:00,21063000.0,21077000.0,21056000.0,21064000.0,1.433399,21066500.0


In [7]:

df_2023['7min_MA'] = df_2023['mid_price'].rolling(window=7).mean()
df_2023['25min_MA'] = df_2023['mid_price'].rolling(window=25).mean()
df_2023['99min_MA'] = df_2023['mid_price'].rolling(window=90).mean()

df_2023['7_ema'] = df_2023['mid_price'].ewm(span=7, adjust=False).mean()
df_2023['25_ema'] = df_2023['mid_price'].ewm(span=25, adjust=False).mean()
df_2023['99_ema'] = df_2023['mid_price'].ewm(span=90,adjust=False).mean()

In [8]:
df_2023 = df_2023.fillna(0)
df_2023.columns

Index(['open', 'high', 'low', 'close', 'volume', 'mid_price', '7min_MA',
       '25min_MA', '99min_MA', '7_ema', '25_ema', '99_ema'],
      dtype='object')

In [75]:
import pandas as pd
import numpy as np


# 변화율 계산 (pct_chg 사용)
df_2023['open_pct_chg'] = df_2023['open'].pct_change()
df_2023['high_pct_chg'] = df_2023['high'].pct_change()
df_2023['low_pct_chg'] = df_2023['low'].pct_change()
df_2023['close_pct_chg'] = df_2023['close'].pct_change()
df_2023['volume_pct_chg'] = df_2023['volume'].pct_change()
df_2023['mid_price_pct_chg'] = df_2023['mid_price'].pct_change()
df_2023['7min_MA_pct_chg'] = df_2023['7min_MA'].pct_change()
df_2023['25min_MA_pct_chg'] = df_2023['25min_MA'].pct_change()
df_2023['99min_MA_pct_chg'] = df_2023['99min_MA'].pct_change()
df_2023['7_ema_pct_chg'] = df_2023['7_ema'].pct_change()
df_2023['25_ema_pct_chg'] = df_2023['25_ema'].pct_change()
df_2023['99_ema_pct_chg'] = df_2023['99_ema'].pct_change()

# 60분 동안의 데이터 추출
df_60min = df_2023.rolling(window=60).mean()

# 필요한 컬럼 선택
selected_columns = df_2023.columns
df_selected = df_2023[selected_columns]

In [76]:
df_2023['open_pct_chg']

index
2023-01-01 09:00:00         NaN
2023-01-01 09:01:00    0.000047
2023-01-01 09:02:00   -0.000854
2023-01-01 09:03:00   -0.000237
2023-01-01 09:04:00    0.000285
                         ...   
2023-08-31 23:55:00   -0.000027
2023-08-31 23:56:00   -0.000568
2023-08-31 23:57:00    0.000000
2023-08-31 23:58:00    0.000000
2023-08-31 23:59:00   -0.000027
Name: open_pct_chg, Length: 344438, dtype: float64

In [77]:
df_selected['open_pct_chg']

index
2023-01-01 09:00:00         NaN
2023-01-01 09:01:00    0.000047
2023-01-01 09:02:00   -0.000854
2023-01-01 09:03:00   -0.000237
2023-01-01 09:04:00    0.000285
                         ...   
2023-08-31 23:55:00   -0.000027
2023-08-31 23:56:00   -0.000568
2023-08-31 23:57:00    0.000000
2023-08-31 23:58:00    0.000000
2023-08-31 23:59:00   -0.000027
Name: open_pct_chg, Length: 344438, dtype: float64

In [85]:
import numpy as np
from tqdm import tqdm


# 주어진 데이터프레임
df = df_selected.iloc[180:]

sequence_length = 64
n_samples = len(df) - sequence_length + 1

# Extract the data as a NumPy array
data_array = df[selected_columns].to_numpy()

# Create an empty array to store the input sequences
input_sequences = np.empty((n_samples, len(selected_columns), sequence_length))

for i in tqdm(range(n_samples)):
    input_seq = data_array[i:i + sequence_length, :].T  # Transpose to have (24, 60) shape
    input_sequences[i] = input_seq

print(input_sequences.shape)


100%|██████████| 344195/344195 [00:02<00:00, 140520.02it/s]

(344195, 24, 64)


In [13]:
(df_60min.iloc[180+sequence_length-1:].index).to_frame().to_csv("train_data_2023_3D_index.csv")

In [14]:
with open('train_data_2023_3D.pkl', 'wb') as f:
    pickle.dump(input_sequences, f)


In [15]:
with open('train_data_2023_3D.pkl', 'rb') as f:
    df = pickle.load(f)

version4를 위해서 차분데이터 생성

In [20]:
import numpy as np
with open('train_data_2023_3D_version4.pkl', 'wb') as f:
    pickle.dump(np.log(input_sequences[:, 12:, :]+2), f) 


In [86]:
np.max(input_sequences[:, 16, :])

32400.47339618303

In [90]:
df_2023['volume_pct_chg'].describe()

count    344437.000000
mean          1.906984
std         101.170285
min          -0.999970
25%          -0.441386
50%          -0.004542
75%           0.790717
max       32400.473396
Name: volume_pct_chg, dtype: float64

In [96]:
np.log(input_sequences[:, 12:])

C:\Users\user\AppData\Local\Temp\ipykernel_12328\3948931006.py:1: RuntimeWarning: divide by zero encountered in log
  np.log(input_sequences[:, 12:])
C:\Users\user\AppData\Local\Temp\ipykernel_12328\3948931006.py:1: RuntimeWarning: invalid value encountered in log
  np.log(input_sequences[:, 12:])


array([[[ -7.55979633,          nan,          nan, ...,         -inf,
                 -inf,          nan],
        [         nan,          nan,  -8.57163403, ...,  -9.95726526,
                  nan,         -inf],
        [        -inf,          nan,         -inf, ...,         -inf,
                 -inf,          nan],
        ...,
        [         nan,          nan,          nan, ..., -10.72467818,
         -11.45721898, -13.11920073],
        [         nan,          nan,          nan, ..., -10.89106923,
         -11.08321945, -11.30081309],
        [-11.29867256, -11.88242606, -11.6360771 , ..., -12.64370541,
         -12.84621268, -13.09407652]],

       [[         nan,          nan,  -7.65505915, ...,         -inf,
                  nan,  -9.95721787],
        [         nan,  -8.57163403,  -9.95811777, ...,          nan,
                 -inf,         -inf],
        [         nan,         -inf,          nan, ...,         -inf,
                  nan,         -inf],
        ...,
